<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initializations" data-toc-modified-id="Initializations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initializations</a></span><ul class="toc-item"><li><span><a href="#Grab-Latest-Model" data-toc-modified-id="Grab-Latest-Model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Grab Latest Model</a></span></li></ul></li><li><span><a href="#Extract-Data" data-toc-modified-id="Extract-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extract Data</a></span><ul class="toc-item"><li><span><a href="#Speed-evaluations" data-toc-modified-id="Speed-evaluations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Speed evaluations</a></span></li><li><span><a href="#Test-Pipeline" data-toc-modified-id="Test-Pipeline-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Test Pipeline</a></span></li></ul></li></ul></div>

Scoping out code needed for prediction pipelines.

# Initializations

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import utils
import discordUtils as du
import schema
import boto3
import pickle
import os
from datetime import datetime, timedelta
from boto3.dynamodb.conditions import  Key, Attr
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import pandas as pd
import sqlUtils as su

os.environ['TZ'] = 'UTC'

In [24]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-2')

cfg_file = utils.findConfig()
cfg = utils.parseConfig(cfg_file)

spark = (
  SparkSession
  .builder
  .appName('redditData')
  .config('spark.driver.extraJavaOptions', '-Duser.timezone=GMT')
  .config('spark.executor.extraJavaOptions', '-Duser.timezone=GMT')
  .config('spark.sql.session.timeZone', 'UTC')
  .config("fs.s3a.access.key", cfg['ACCESSKEY'])
  .config("fs.s3a.secret.key", cfg['SECRETKEY'])
  .getOrCreate()
)

## Grab Latest Model

In [66]:
s3 = boto3.resource('s3', region_name='us-east-2')
bucketName = 'data-kennethmyers'
bucket = s3.Bucket(bucketName)
objs = bucket.objects.filter(Prefix='models/Reddit_LR_model_')
latestModelLoc = sorted([obj.key for obj in objs])[-1]
print(f"Latest model location: s3a://{bucketName}/{latestModelLoc}")
modelSaveLoc = './pickledModels/latestModel.sav'
s3_client = boto3.client('s3', region_name='us-east-2')
s3_client.download_file('data-kennethmyers', latestModelLoc, modelSaveLoc)
model = pickle.load(open(modelSaveLoc, 'rb'))

Latest model location: s3a://data-kennethmyers/models/Reddit_LR_model_20230414-061010.sav


# Extract Data

## Speed evaluations

When I wrote the model ETL process, the data was read to Spark DataFrames so that the process could be extended to distributed systems. During the production predictive ETL process, we don't need as much data, in fact there are typically only a few to predict on. As such, we may not need spark for this process and we can use something lighter. 

Here I'm going to compare the speed of loading the data to spark vs pandas. We could actually get away with something lighter than pandas (ie. keep it in dictionary format) but pandas can help with loading to sql later.

In [ ]:
now = datetime.utcnow()
tenMinAgo = now-timedelta(seconds=600)
tenMinAgoDate = tenMinAgo.strftime('%Y-%m-%d') 
tenMinAgoTime = tenMinAgo.strftime('%H:%M:%S') 

risingTable = dynamodb_resource.Table('rising')

postIdQueryResult = risingTable.query(
    IndexName='byLoadDate',
    KeyConditionExpression=Key('loadDateUTC').eq(tenMinAgoDate) & Key('loadTimeUTC').gte(tenMinAgoTime),
    FilterExpression=Attr('timeElapsedMin').eq(60),
    ProjectionExpression='postId'
  )['Items']
postsOfInterest = {res['postId'] for res in postIdQueryResult}


In [131]:
%%timeit
postIdData = utils.getPostIdSparkDataFrame(spark, risingTable, postsOfInterest, chunkSize=100)
pandasTestDf = postIdData.toPandas()


1.27 s ± 35.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
print(pandasTestDf.head().to_string())

  loadDateUTC         loadTimeUTC           loadTSUTC   postId subreddit                                                                              title        createdTSUTC  timeElapsedMin  score  numComments  upvoteRatio  numGildings
0  2023-04-19 1900-01-01 01:33:21 2023-04-19 01:33:21  12raffr      pics                                           The light through this glass of mint tea 2023-04-19 00:32:35              60     16            3         0.95            0
1  2023-04-19 1900-01-01 01:40:21 2023-04-19 01:40:21  12ramm9      pics                                                     They aren’t ready for my Queen 2023-04-19 00:39:54              60     47            4         0.87            0
2  2023-04-19 1900-01-01 01:33:21 2023-04-19 01:33:21  12rag30      pics                                                                  Ye olde drag show 2023-04-19 00:33:14              60      1            2         0.54            0
3  2023-04-19 1900-01-01 01:30:21 2023-04-19 01:

In [133]:
pandasTestDf.dtypes

loadDateUTC       datetime64[ns]
loadTimeUTC       datetime64[ns]
loadTSUTC         datetime64[ns]
postId                    object
subreddit                 object
title                     object
createdTSUTC      datetime64[ns]
timeElapsedMin             int32
score                      int32
numComments                int32
upvoteRatio              float32
numGildings                int32
dtype: object

In [134]:
%%timeit
postIdData = utils.getPostIdPdDataFrame(risingTable, postsOfInterest, chunkSize=100)

993 ms ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [135]:
print(postIdData.head().to_string())

         createdTSUTC         loadTimeUTC  upvoteRatio  timeElapsedMin  numComments loadDateUTC subreddit  score   postId           loadTSUTC  numGildings                                                                              title
0 2023-04-19 00:32:35 1900-01-01 01:33:21         0.95              60            3  2023-04-19      pics     16  12raffr 2023-04-19 01:33:21            0                                           The light through this glass of mint tea
1 2023-04-19 00:39:54 1900-01-01 01:40:21         0.87              60            4  2023-04-19      pics     47  12ramm9 2023-04-19 01:40:21            0                                                     They aren’t ready for my Queen
2 2023-04-19 00:33:14 1900-01-01 01:33:21         0.54              60            2  2023-04-19      pics      1  12rag30 2023-04-19 01:33:21            0                                                                  Ye olde drag show
3 2023-04-19 00:29:39 1900-01-01 01:30:21       

In [136]:
pandasTestDf.dtypes

loadDateUTC       datetime64[ns]
loadTimeUTC       datetime64[ns]
loadTSUTC         datetime64[ns]
postId                    object
subreddit                 object
title                     object
createdTSUTC      datetime64[ns]
timeElapsedMin             int32
score                      int32
numComments                int32
upvoteRatio              float32
numGildings                int32
dtype: object

In many situations, a 20% speed improvement can be significant (ie a streaming setting). We are running periodic batch jobs so it is not especially important but I wanted to demonstrate how changing the process between modeling and predicting can provide improvements. 

For now, I don't want to rewrite my transformation processes in pandas, so I will continue to use the spark process.

## Test Pipeline

In [77]:
# originally was defining here but later moved to python file
from PredictETL import Pipeline
from tests import testPipeline

testPipeline(model, spark, threshold=0.0400)

    postId subreddit  title        createdTSUTC  maxScore20m  maxScore21_40m  maxScore41_60m  maxNumComments20m  maxNumComments21_40m  maxNumComments41_60m  maxUpvoteRatio20m  maxUpvoteRatio21_40m  maxUpvoteRatio41_60m  maxNumGildings20m  maxNumGildings21_40m  maxNumGildings41_60m  maxScoreGrowth21_40m41_60m  maxNumCommentsGrowth21_40m41_60m  predict_proba_1  stepUp
0  1234567      pics  Test1 2023-04-19 03:14:30            0               0              21                  0                     0                     1               0.00                  0.00                  0.97                  0                     0                     0                         0.0                               0.0         0.023970       0
1  1234568      pics  Test2 2023-04-20 03:14:30           10              29              67                  1                     6                     9               0.95                  0.95                  0.94                  0                     0   

True

In [78]:
# testing upsert with a simple example

from sqlalchemy import create_engine
import pandas as pd
import sqlUtils as su

indexes = ["id1","id2"]
testDf  = pd.DataFrame(
  {
    "id1": [1, 2, 3, 4, 5],
    "id2": ["a", "a", "b", "c", "d"],
    "name": ["surname1", "surname2", "surname3", "name4", "dname"],
    "age": [13, 44, 29, 68, 100],
  }
).set_index(indexes)

engine = create_engine(f"postgresql+pg8000://{cfg['USERNAME']}:{cfg['PASSWORD']}@{cfg['HOST']}:{cfg['PORT']}/{cfg['DATABASE']}")

with engine.connect() as conn:
  result = su.upsert_df(df=testDf, table_name='test_upsert_table', engine=conn)
  print('completed upsert test')

completed upsert test
